# Transect

Groupby and transform allow me to combine rows into a single 'transect' row.
Or, use a multiIndex, a hierarchical index, so I can target specific cells using id and type. The index item for a multiIndex is a TUPLE.

In [1]:
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from io import StringIO
import re
print(sys.version)
print("Pandas:", pd.__version__)

3.6.0 |Anaconda custom (64-bit)| (default, Dec 23 2016, 11:57:41) [MSC v.1900 64 bit (AMD64)]
Pandas: 0.19.2


In [11]:
df = pd.read_excel('C:/Users/Peter/Documents/atlas/atlasdata/ready/missed_transects.xls')
#df.date = df.date.astype(dtype='str')# we don't need numbers in this dataset.
#df=df.replace('nan','')
#df=df.replace(' ','')
#this turns dates into strings with the proper format for JSON:
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.shape

(33, 10)

In [12]:
g = df.groupby(['name','date'])
len(g)

16

In [13]:
tr_cols = ['mapPhoto','linephoto1','linephoto2','miscPhoto','general_observations','site_description']
for col in tr_cols:
    df[col] = df[col].replace(np.nan,'')
    df[col]=g[col].transform(lambda x: "%s" % ''.join(x))

In [14]:
df.to_excel('ready/missed_transects1.xls', index=False)

In [10]:
df.to_csv('obs_types/transect_test.csv',index=False)

In [76]:
df.loc[df.type == 'plants',['general_observations']]=df['note']

In [47]:
df['site_description'].replace(np.nan,'')
df['caption'].replace(np.nan,'')
# gotta have else statement if using if with lambda
df['site_description'] = df.apply(lambda row: str(row['site_description']) + ' | ' + str(row['caption']) if (row.type=='transect' or row.type=='remonitoring notes') else '', axis=1)


In [56]:
df = df[(df.type=='transect') | (df.type=='remonitoring notes')]

In [58]:
df.to_excel('obs_types/transect_5.xls', index=False)

In [80]:
df = df[['id','date','type','mapPhoto','linephoto1','linephoto2','miscPhoto','site_description','general_observations','observers']]


In [83]:
df.to_csv('obs_types/transect_test.csv', index=False)

In [52]:
df.to_csv('C:/Users/Peter/Documents/atlas/atlasdata/obs_types/transect_new1.csv', index=False)

In [8]:
grouped = df.groupby(['id','date'])
mykeys = grouped.groups.keys() # this gives a dict_key object
#myvalues = grouped.groups.values

## Captions for photos

### Need to aggregate captions before "photo position, notes" is added. Then add these to site_description along with the Note field.

In [36]:

# gotta have else statement if using if with lambda
df['caption'] = df.apply(lambda row: ('from ' + row.position + ', ' + row.note + ' | ') if row.newurl !='' and (row.position!='' or row.note != '') else '', axis=1)


In [46]:

# gotta have else statement if using if with lambda
df['caption'] = df.apply(lambda row: 'photos: ' + row.caption if row.caption !='' else '', axis=1)


In [22]:
df.site_description[45]

'Vegetable production. Ripped in Feb 2015. No worms seen. 8percent SW slope.'

In [118]:
df['position'] = df.type.str.findall('\d+') #finds all digits in df.type
df.position = df.position.apply(lambda x: "%s" % ''.join(x)) #turns list into string

## Labeling 'photo' type
because it's not specified which direction line photos are taken. Make an arbitrary choice. Using loc for assignment: df.loc[destination condition, column] = df.loc[source]

## Check for too many photos in one category

In [126]:
#assign uncategorized photos to a category
mycols = ['linephoto1','linephoto2','miscPhoto','extraPhoto','extraPhoto','extraPhoto']
def munge_photo_type():
    for item in list(mykeys):
        g = grouped.get_group(item) # this is a dataframe
        i = g.index
        #print(g.loc[i,'type'])
        for x in range(len(i)):
            #print(i[x],df.loc[i[x],mycols[x]])
            df2.loc[i[x],mycols[x]]=df2.loc[i[x],'newurl']
            
        
munge_photo_type()

    

In [33]:

for item in list(mykeys):
    g = grouped.get_group(item)
    if len(g.miscPhoto.unique())>2:
        print (item, len(g.miscPhoto.unique()))

In [129]:
df.loc[df.type=='map',['mapPhoto']]=df['newurl']
df.loc[df.type.str.contains('lineminus'),['miscPhoto']]=df['newurl']
df.loc[df.type.str.contains('lineplus'),['miscPhoto']]=df['newurl']
df.loc[df.type.str.contains('misc'),['miscPhoto']]=df['newurl']
df.loc[df.type.str.contains('line-'),['linephoto1']]=df['newurl']
df.loc[df.type.str.contains('line\+'),['linephoto2']]=df['newurl']#be sure to escape the +

In [130]:
df.loc[df.type =='transect',['site_description']]= df['note']
df.loc[df.type =='remonitoring notes',['site_description']]= df['note']

In [132]:
df = df[['group',
 'id',
 'date',
 'type',
 'note',
 'mapPhoto',
 'linephoto1',
 'linephoto2',
 'miscPhoto',
 'site_description',
 'general_observations',
 'position',
 'observers',
 'url',
 'newurl']]

In [135]:
df.to_csv('/Users/Peter/Documents/atlas/atlasdata/obs_types/transectNEW.csv',index=False)

## use groupby and transform to fill the row

In [9]:
#since we're using string methods, NaNs won't work
mycols =['general_observations','mapPhoto','linephoto1','linephoto2','miscPhoto','site_description']
for item in mycols:
    df[item] = df[item].fillna('')

In [40]:
df['site_description'] = df['site_description'].str.strip()

In [42]:
df['caption'] = df.groupby(['id','date'])['caption'].transform(lambda x: "%s" % ''.join(x))

In [45]:
print(df.loc[159,'caption'])

from 48, east along transect | from 48, west along transect | from 48, west along transect |


In [18]:
df.to_csv('test.csv')
#done to here. Next, figure out what to do with linephotos, unclassified photos, and their notes.
#make column for photocaptions. When adding linephoto1, add 'note' and 'type' fields to caption column. E.g. 'linephoto1: 100line- | view east along transect.' Then join the rows in the groupby transform and add to site_description field.

In [172]:
df.shape

(1643, 25)

In [23]:
df[(df.type.str.contains('line\+'))&(df.linephoto2.str.len()<50)]

,group,id,date,type,note,mapPhoto,linephoto1,linephoto2,miscPhoto,site_description,general_observations,position,observers,url,newurl,caption
26,ANTK1,ANTK1,2014-04-23,25line+,south along transect,observations/tomk_ANTK1map.jpg,observations/tomk_antk-001.jpg,observations/tomk_antk-003.jpg,observations/tomk_antk-002.jpg,NaN,,25.0,NaN,http://soilcarboncoalition.org/data/tomk/antk ...,observations/tomk_antk-003.jpg,"photos: from 25, north along transect | from 2..."
31,ANTK1,ANTK1,2017-05-04,25line+,NaN,,observations/tomk_ANTK1-25line-.jpg,observations/tomk_ANTK1-25line+.jpg,,NaN,,25.0,NaN,http://soilcarboncoalition.org/data/tomk/ANTK1...,observations/tomk_ANTK1-25line+.jpg,photos: from 25 | from 25 |
35,ANTK2,ANTK2,2014-04-24,30line+,north along transect,observations/tomk_ANTK2map.jpg,observations/tomk_antk-006.jpg,observations/tomk_antk-007.jpg,,NaN,,30.0,NaN,http://soilcarboncoalition.org/data/tomk/antk ...,observations/tomk_antk-007.jpg,"photos: from 30, south along transect | from 3..."
39,ANTK2,ANTK2,2017-05-04,30line+,NaN,,observations/tomk_ANTK2-30line-.jpg,observations/tomk_ANTK2-30line+.jpg,,NaN,,30.0,NaN,http://soilcarboncoalition.org/data/tomk/ANTK2...,observations/tomk_ANTK2-30line+.jpg,photos: from 30 | from 30 |
42,ANTK3,ANTK3,2015-04-15,line+,NaN,observations/tomk_ANTK3map.jpg,observations/tomk_ANTK3line-.jpg,observations/tomk_ANTK3line+.jpg,,NaN,,NaN,NaN,http://soilcarboncoalition.org/data/tomk/ANTK3...,observations/tomk_ANTK3line+.jpg,photos: peak in middle is 66 deg. Magnetic fro...
47,ANTK4,ANTK4,2015-04-16,30line+,NaN,observations/tomk_ANTK4map.jpg,observations/tomk_ANTK4-30line-.jpg,observations/tomk_ANTK4-30line+.jpg,,NaN,,30.0,NaN,http://soilcarboncoalition.org/data/tomk/ANTK4...,observations/tomk_ANTK4-30line+.jpg,photos: from 30 | from 30 | from 30 |
52,ARRI1,ARRI1,2012-03-14,26line+,NNE along transect,observations/arri_AR1map.jpg,observations/arri_Mar18-2012-005.jpg,observations/arri_Mar18-2012-004.jpg,,NaN,"orchardgrass, smooth brome, alfalfa, plum, sum...",26.0,NaN,http://soilcarboncoalition.org/data/arri/Mar18...,observations/arri_Mar18-2012-004.jpg,"photos: from 26, SSW along transect | from 26,..."
57,AVSP1,AVSP1,2011-05-01,20line+,looking east from bucket lid,observations/avsp_AV1map.jpg,observations/avsp_may1-2011-028.jpg,observations/avsp_may1-2011-029.jpg,observations/avsp_may1-2011-034.jpg,NaN,,20.0,NaN,http://soilcarboncoalition.org/data/avsp/may1-...,observations/avsp_may1-2011-029.jpg,"photos: from 20, looking west from bucket lid ..."
61,BCFM1,BCFM1,2015-06-25,30line+,NaN,observations/bcfm_BCFM1map.jpg,,observations/bcfm_BCFM1-30line+.jpg,,NaN,,30.0,NaN,http://soilcarboncoalition.org/data/bcfm/BCFM1...,observations/bcfm_BCFM1-30line+.jpg,photos: from 30 |
66,BCLA1,BCLA1,2012-01-06,50line+,north along transect,observations/bcla_BC1map.jpg,observations/bcla_Jan20-2012-010.jpg,observations/bcla_Jan20-2012-012.jpg,observations/bcla_Jan20-2012-011.jpg,NaN,"blue grama, big bluestem, sideoats grama, japa...",50.0,NaN,http://soilcarboncoalition.org/data/bcla/Jan20...,observations/bcla_Jan20-2012-012.jpg,"photos: from 50, south along transect | from 5..."


In [76]:
df.linephoto2.str.len().sort_values(ascending=False)
#print(df.linephoto2[1046])

1043    91
1039    91
1046    91
1040    91
56      70
57      70
58      70
59      70
97      70
96      70
95      70
94      70
54      70
93      70
92      70
586     70
585     70
584     70
583     70
582     70
55      70
196     68
194     68
192     68
197     68
201     68
200     68
199     68
198     68
1150    60
        ..
404      0
850      0
861      0
870      0
911      0
912      0
457      0
456      0
455      0
1319     0
430      0
1006     0
1007     0
1008     0
403      0
282      0
402      0
401      0
400      0
399      0
398      0
397      0
396      0
395      0
394      0
393      0
313      0
312      0
311      0
0        0
Name: linephoto2, dtype: int64

## shift data to correct row using a multi-Index

In [280]:
ids = list(df['id'])#make a list of ids to iterate over, before the hierarchical index

In [281]:
#df.type = df.type.map({'\*plot summary':'transect','\*remonitoring notes':'transect'})

In [288]:
df.loc[df.type =='map',['mapPhoto']]=df['url'] #moving cell values to correct column

In [274]:
df.set_index(['id','type'],inplace=True) # hierarchical index so we can call locations

In [276]:
#a hierarchical index uses a tuple. You can set values using loc.
#this format: df.loc[destination] = df.loc[source].values[0]
for item in ids:
    df.loc[(item,'*plot summary'),'mapPhoto'] = df.loc[(item,'map'),'mapPhoto'].values[0]

#generates a pink warning about performance, but oh well.

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1310: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelbase.py:390: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\zmqshell.py:501: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [10]:
#here we are using an expression in parens to test for a condition
(df['type'].str.contains('\s') & df['note'].notnull()).value_counts()

False    24
True      6
dtype: int64

In [162]:
df.url = df.url.str.replace(' ','_');df.url

0           tomk_antk_003.jpg
1           tomk_antk_001.jpg
2           tomk_antk_002.jpg
3           tomk_antk_003.jpg
4           tomk_ANTK1map.jpg
5           tomk_antk_007.jpg
6           tomk_antk_006.jpg
7           tomk_antk_007.jpg
8           tomk_ANTK2map.jpg
9         tomk_ANTK3line-.jpg
10        tomk_ANTK3line-.jpg
11        tomk_ANTK3line+.jpg
12          tomk_linemisc.jpg
13          tomk_ANTK3map.jpg
14     tomk_ANTK4-30line+.jpg
15     tomk_ANTK4-30line-.jpg
16     tomk_ANTK4-30line+.jpg
17    tomk_ANTK4-30line++.jpg
18          tomk_ANTK4map.jpg
19                        NaN
20     tomk_ANTK1-25line-.jpg
21     tomk_ANTK1-25line+.jpg
22                        NaN
23     tomk_ANTK2-30line-.jpg
24     tomk_ANTK2-30line+.jpg
25     avsp_may1-2011_028.jpg
26     avsp_may1-2011_029.jpg
27            avsp_AV1map.jpg
28     avsp_may1-2011_033.jpg
29     avsp_may1-2011_034.jpg
Name: url, dtype: object

In [151]:
df.url.head()

0    http://soilcarboncoalition.org/data/tomk/antk ...
1    http://soilcarboncoalition.org/data/tomk/antk ...
2    http://soilcarboncoalition.org/data/tomk/antk ...
3    http://soilcarboncoalition.org/data/tomk/antk ...
4    http://soilcarboncoalition.org/data/tomk/ANTK1...
Name: url, dtype: object

In [ ]:
df['newurl'] = df.url.str.replace

In [130]:
df.newurl.head()

0    tomk_antk_003.jpg
1    tomk_antk_001.jpg
2    tomk_antk_002.jpg
3    tomk_antk_003.jpg
4    tomk_ANTK1map.jpg
Name: newurl, dtype: object

In [ ]:
#for combining rows try something like this:
print(df.groupby('somecolumn')['temp variable'].apply(' '.join).reset_index())